In [13]:
from pathlib import Path
from tqdm import tqdm

FEATURES_DIR = Path("/data/histaug/features")

In [21]:
to_delete = []


def delete(dir: Path):
    to_delete.append(dir)


for dataset_dir in FEATURES_DIR.iterdir():
    if not dataset_dir.is_dir() or dataset_dir.name == "kather100k":
        continue

    i = 0
    for feature_extractor_dir in tqdm(list(dataset_dir.iterdir()), desc=f"Processing {dataset_dir.name}"):
        if not feature_extractor_dir.is_dir() or feature_extractor_dir.name in ("owkin_teacher", "mocov2"):
            continue

        for slide_dir in feature_extractor_dir.glob("*.zarr"):
            if not slide_dir.is_dir():
                continue

            feats_augs_dir = slide_dir / "feats_augs"
            if feats_augs_dir.exists():
                for aug_dir in feats_augs_dir.iterdir():
                    if not aug_dir.is_dir():
                        continue
                    if aug_dir.name != "Macenko":
                        delete(aug_dir)
                        i += 1
    print(f"Found {i} directories to delete in {dataset_dir.name}")

Processing CPTAC-BRCA: 100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


Found 0 directories to delete in CPTAC-BRCA


Processing CAMELYON17: 100%|██████████| 11/11 [00:34<00:00,  3.13s/it]


Found 130000 directories to delete in CAMELYON17


Processing TCGA-BRCA:  92%|█████████▏| 11/12 [02:09<00:13, 13.62s/it]

In [ ]:
with open("delete.sh", "w") as f:
    for i, dir in enumerate(to_delete):
        if i % 1000 == 0:
            f.write(f'echo "{i} / {len(to_delete)} ({i / len(to_delete) * 100:.2f}%)"\n')
        f.write(f'rm -rf "{dir}"\n')